In [ ]:
# Welcome to Langchain Deep Dive

In [1]:
# Vectors Srore - FAISS, Vector DB - Chroma DB , PineCone etc

In [2]:
"""plug in a GPT model and do a tiny, clean RAG (Retrieval‑Augmented Generation): retrieve the most relevant snippets from your FAISS index and ask the model to answer using only that context.

Below is a single Colab‑ready code block that:

builds a small FAISS index (you can paste your own text),

wraps it as a retriever,

formats retrieved context,

calls a GPT model to answer from that context,

runs an interactive Q&A loop.
"""


"""
What this gives us:

Embeddings + FAISS turn your reference text into a searchable index.

A retriever fetches the top-k relevant chunks for each question.

A prompt instructs the model to answer only from the retrieved context, avoiding hallucinations.

A tiny interactive loop to ask multiple questions.
"""


'\nWhat this gives us:\n\nEmbeddings + FAISS turn your reference text into a searchable index.\n\nA retriever fetches the top-k relevant chunks for each question.\n\nA prompt instructs the model to answer only from the retrieved context, avoiding hallucinations.\n\nA tiny interactive loop to ask multiple questions.\n'

In [ ]:
# Example 1 - Chunking + Metadata + Citations

In [6]:
# --- Install ---
!pip install -q langchain-openai langchain-community langchain-text-splitters faiss-cpu

# --- Imports & API key ---
from langchain_openai import OpenAIEmbeddings, ChatOpenAI
from langchain_text_splitters import RecursiveCharacterTextSplitter
from langchain_community.vectorstores import FAISS
from langchain_core.prompts import ChatPromptTemplate
from langchain_core.output_parsers import StrOutputParser
from langchain_core.runnables import RunnablePassthrough
from google.colab import userdata
from langchain.schema import Document
import os

os.environ["OPENAI_API_KEY"] = userdata.get("OPENAI_API_KEY")

# --- 1) Source text (replace with your own) ---
raw_text = """
Mahendra Singh Dhoni (MSD) captained India and won the 2007 T20 World Cup and the 2011 ODI World Cup.
Sachin Ramesh Tendulkar (SRT) is widely called the 'God of Cricket' for his batting records.
Amitabh Bachchan, nicknamed Big B, is a legendary actor in Indian cinema.
Shah Rukh Khan, often called King Khan, is one of the most popular Bollywood actors worldwide.
Virat Kohli is renowned for batting consistency across formats.
Rohit Sharma is known for explosive batting and captaincy in limited-overs cricket.
"""

# --- 2) Chunk with metadata (source + chunk_id) ---
splitter = RecursiveCharacterTextSplitter(chunk_size=280, chunk_overlap=80)
chunks = splitter.split_text(raw_text)
docs = [
    Document(page_content=chunk, metadata={"source": "notes.txt", "chunk_id": i})
    for i, chunk in enumerate(chunks)
]

# --- 3) Build vector store + retriever ---
emb = OpenAIEmbeddings(model="text-embedding-3-small")
vs = FAISS.from_documents(docs, embedding=emb)
retriever = vs.as_retriever(search_kwargs={"k": 3})

# --- 4) Prompt that asks model to cite chunks used ---
prompt = ChatPromptTemplate.from_template(
    "You are a helpful assistant. Use ONLY the context to answer.\n\n"
    "Context:\n{context}\n\n"
    "Question: {question}\n\n"
    "If the answer is not in the context, say you don't know.\n"
    "At the end, list citations as [source#chunk_id] for each chunk you used."
)

def format_docs(docs):
    # Include metadata so the model can cite properly
    lines = []
    for d in docs:
        src = d.metadata.get("source", "unknown")
        cid = d.metadata.get("chunk_id", "na")
        lines.append(f"[{src}#{cid}] {d.page_content}")
    return "\n\n".join(lines)

llm = ChatOpenAI(model="gpt-4o-mini", temperature=0.2)
rag_chain = (
    {"context": retriever | format_docs, "question": RunnablePassthrough()}
    | prompt
    | llm
    | StrOutputParser()
)

print("Ask questions (type 'exit' to quit):")
while True:
    q = input("Q: ").strip()
    if q.lower() in ["exit", "quit"]:
        print("👋 Goodbye!")
        break
    print("A:", rag_chain.invoke(q), "\n")


Ask questions (type 'exit' to quit):
Q: who is Virat Kohli
A: Virat Kohli is renowned for batting consistency across formats. [source#2] 

Q: who is SRT
A: SRT refers to Sachin Ramesh Tendulkar, who is widely called the 'God of Cricket' for his batting records. 

Citations: [notes.txt#0] 

Q: WHO IS srk
A: SRK refers to Shah Rukh Khan, who is often called King Khan and is one of the most popular Bollywood actors worldwide. 

Citations: [notes.txt#1] 

Q: who is virendra sehwag
A: I don't know. 

Citations: [notes.txt#2] 

Q: exit
👋 Goodbye!


In [7]:
# Example 2 - Persistence (Save/Load FAISS)

In [8]:
# ================================
# FAISS persistence
# ================================

# 0) Install
!pip install -q langchain-openai langchain-community faiss-cpu

# 1) Imports & API key
from langchain_openai import OpenAIEmbeddings
from langchain_community.vectorstores import FAISS
from google.colab import userdata
import os, sys

api_key = userdata.get("OPENAI_API_KEY")
if not api_key:
    print("❌ OPENAI_API_KEY not found.\n"
          "Set it once with:\n"
          "from google.colab import userdata\n"
          "userdata.set('OPENAI_API_KEY', 'sk-...')\n")
    sys.exit(1)
os.environ["OPENAI_API_KEY"] = api_key

# 2) Small corpus
texts = [
    "MSD is Mahendra Singh Dhoni, a legendary Indian cricket captain.",
    "MSD won multiple world cups for India"
    "MSD is the first captain ever to win a T20 world cup"
    "SRT refers to Sachin Ramesh Tendulkar, the 'God of Cricket'.",
    "Amitabh Bachchan is nicknamed Big B in Indian cinema.",
    "Shah Rukh Khan is popularly called King Khan.",
    "Virat Kohli is known for batting consistency across formats.",
    "Rohit Sharma is known for explosive batting in limited-overs."
]

# 3) Build vector store (embeddings + FAISS)
emb = OpenAIEmbeddings(model="text-embedding-3-small")
vs = FAISS.from_texts(texts, embedding=emb)
print("✅ Built FAISS index with", len(texts), "texts.")

# 4) Save to disk (persistence)
save_dir = "/content/my_faiss_plain"
vs.save_local(save_dir)
print("💾 Saved index to:", save_dir)

# 5) Load it back (later or in a new runtime)
vs2 = FAISS.load_local(save_dir, emb, allow_dangerous_deserialization=True)
print("🔁 Reloaded index. Size =", len(vs2.index_to_docstore_id))

# 6) Plain similarity search (no MMR)
retriever_plain = vs2.as_retriever(search_kwargs={"k": 3})  # default: top-k similarity

def search(query: str, k: int = 3):
    docs = retriever_plain.get_relevant_documents(query)
    print(f"\n🔎 Top-{k} results for: {query!r}")
    for i, d in enumerate(docs[:k], 1):
        print(f"{i}. {d.page_content}")

# 7) Try a couple of queries
search("Who is King Khan?", k=3)
search("Who is MSD?", k=3)
search("Who is called the God of Cricket?", k=3)



✅ Built FAISS index with 6 texts.
💾 Saved index to: /content/my_faiss_plain
🔁 Reloaded index. Size = 6


/tmp/ipython-input-1382641590.py:53: LangChainDeprecationWarning: The method `BaseRetriever.get_relevant_documents` was deprecated in langchain-core 0.1.46 and will be removed in 1.0. Use :meth:`~invoke` instead.
  docs = retriever_plain.get_relevant_documents(query)



🔎 Top-3 results for: 'Who is King Khan?'
1. Shah Rukh Khan is popularly called King Khan.
2. Amitabh Bachchan is nicknamed Big B in Indian cinema.
3. Virat Kohli is known for batting consistency across formats.

🔎 Top-3 results for: 'Who is MSD?'
1. MSD is Mahendra Singh Dhoni, a legendary Indian cricket captain.
2. MSD won multiple world cups for IndiaMSD is the first captain ever to win a T20 world cupSRT refers to Sachin Ramesh Tendulkar, the 'God of Cricket'.
3. Rohit Sharma is known for explosive batting in limited-overs.

🔎 Top-3 results for: 'Who is called the God of Cricket?'
1. MSD won multiple world cups for IndiaMSD is the first captain ever to win a T20 world cupSRT refers to Sachin Ramesh Tendulkar, the 'God of Cricket'.
2. MSD is Mahendra Singh Dhoni, a legendary Indian cricket captain.
3. Virat Kohli is known for batting consistency across formats.


In [9]:
# Example 3 - RAG Implementation - supporting PDF / URL / TXT with a simple switch.

"""
It:

loads the document(s)

chunks the text

builds a FAISS index (cosine similarity)

runs a small RAG loop with citations
"""


'\nIt:\n\nloads the document(s)\n\nchunks the text\n\nbuilds a FAISS index (cosine similarity)\n\nruns a small RAG loop with citations\n'

In [13]:
# =========================================
# RAG over PDF / URL / TXT
# =========================================

# 0) Install
!pip install -q langchain-openai langchain-community langchain-text-splitters faiss-cpu pypdf bs4

# 1) Imports & API key
from langchain_openai import OpenAIEmbeddings, ChatOpenAI
from langchain_community.vectorstores import FAISS
from langchain_community.vectorstores.utils import DistanceStrategy
from langchain_text_splitters import RecursiveCharacterTextSplitter
from langchain_community.document_loaders import PyPDFLoader, TextLoader, WebBaseLoader
from langchain_core.prompts import ChatPromptTemplate
from langchain_core.output_parsers import StrOutputParser
from langchain_core.runnables import RunnablePassthrough
from langchain.schema import Document
from google.colab import userdata
import os

# API key
os.environ["OPENAI_API_KEY"] = userdata.get("OPENAI_API_KEY")

# 2) === Choose your source here ===
#SOURCE_TYPE = "txt"   # "pdf" | "url" | "txt"
SOURCE_TYPE = "url"
#SOURCE_TYPE = "pdf"

# Provide ONE of these depending on SOURCE_TYPE
PDF_PATH = "/content/Databricks-Big-Book-Of-GenAI-FINAL.pdf"                       # e.g., upload via Colab sidebar, then set path
URLS = ["https://docs.databricks.com/aws/en/machine-learning/"]  # example URL list
TXT_PATH = "/content/notes.txt"
                     # plain text file

# 3) Load documents
def load_docs():
    if SOURCE_TYPE == "pdf":
        loader = PyPDFLoader(PDF_PATH)
        return loader.load()
    elif SOURCE_TYPE == "url":
        loader = WebBaseLoader(URLS)
        return loader.load()
    elif SOURCE_TYPE == "txt":
        loader = TextLoader(TXT_PATH, encoding="utf-8")
        return loader.load()
    else:
        raise ValueError("Unsupported SOURCE_TYPE. Use 'pdf', 'url', or 'txt'.")

docs = load_docs()
print(f"Loaded {len(docs)} document chunks (pre-split).")

# 4) Split into chunks (tune if needed)
splitter = RecursiveCharacterTextSplitter(chunk_size=800, chunk_overlap=120)
splits = splitter.split_documents(docs)
print(f"Split into {len(splits)} chunks.")

# 5) Build vector store with cosine similarity
emb = OpenAIEmbeddings(model="text-embedding-3-small")
vs = FAISS.from_documents(
    splits, embedding=emb, distance_strategy=DistanceStrategy.COSINE
)
retriever = vs.as_retriever(search_kwargs={"k": 3})
print("Vector store ready.")

# 6) Prompt with “use context only” + citations
prompt = ChatPromptTemplate.from_template(
    "You are a helpful assistant. Use ONLY the context to answer.\n\n"
    "Context:\n{context}\n\n"
    "Question: {question}\n\n"
    "If the answer is not in the context, say you don't know.\n"
    "At the end, list citations as [source#page_or_id]."
)

def format_docs(docs):
    lines = []
    for d in docs:
        src = d.metadata.get("source") or d.metadata.get("url") or "unknown"
        page = d.metadata.get("page", d.metadata.get("chunk_id", "na"))
        lines.append(f"[{src}#{page}] {d.page_content}")
    return "\n\n".join(lines)

# 7) LLM + chain
llm = ChatOpenAI(model="gpt-4o-mini", temperature=0.2)
rag_chain = (
    {"context": retriever | format_docs, "question": RunnablePassthrough()}
    | prompt
    | llm
    | StrOutputParser()
)

# 8) (Optional) Save for reuse later
SAVE_DIR = "/content/my_faiss_index_docs"
vs.save_local(SAVE_DIR)
print("Saved FAISS index to:", SAVE_DIR)

# 9) Q&A loop
print("\n💬 Ask about your document(s). Type 'exit' to quit.")
while True:
    q = input("Q: ").strip()
    if q.lower() in ["exit", "quit"]:
        print("👋 Goodbye!")
        break
    ans = rag_chain.invoke(q)
    print("A:", ans, "\n")


Loaded 118 document chunks (pre-split).
Split into 291 chunks.
Vector store ready.
Saved FAISS index to: /content/my_faiss_index_docs

💬 Ask about your document(s). Type 'exit' to quit.
Q: what is dbrx
A: DBRX is a transformer-based decoder-only large language model (LLM) that was trained using next-token prediction. It employs a fine-grained mixture-of-experts (MoE) architecture with 132 billion total parameters, of which 36 billion parameters are active on any input. DBRX was pre-trained on 12 trillion tokens of text and code data. It features 16 experts and selects 4, providing 65 times more possible combinations of experts compared to other models like Mixtral and Grok-1. Additionally, DBRX utilizes rotary position encodings (RoPE), gated linear units (GLU), and grouped query attention (GQA) [source#6]. 

Q: what is databricks
A: Databricks is the data and AI company that provides a platform for unifying and democratizing data, analytics, and AI. It is relied upon by more than 10,0

In [14]:
# RAG over PDF/URL/TXT using Chroma DB (Vector Database)

In [18]:
# =========================================
# RAG over PDF / URL / TXT using Chroma DB
# =========================================

# 0) Install dependencies
!pip install -q langchain-openai langchain-community langchain-text-splitters chromadb pypdf bs4

# 1) Imports & API Key
from langchain_openai import OpenAIEmbeddings, ChatOpenAI
from langchain_community.vectorstores import Chroma
from langchain_text_splitters import RecursiveCharacterTextSplitter
from langchain_community.document_loaders import PyPDFLoader, TextLoader, WebBaseLoader
from langchain_core.prompts import ChatPromptTemplate
from langchain_core.output_parsers import StrOutputParser
from langchain_core.runnables import RunnablePassthrough
from langchain.schema import Document
from google.colab import userdata
import os

# API key
os.environ["OPENAI_API_KEY"] = userdata.get("OPENAI_API_KEY")

# 2) === Choose your source here ===
# SOURCE_TYPE = "txt"   # "pdf" | "url" | "txt"
SOURCE_TYPE = "url"
#SOURCE_TYPE = "pdf"

# Provide ONE of these depending on SOURCE_TYPE
PDF_PATH = "/content/Databricks-Big-Book-Of-GenAI-FINAL.pdf"
URLS = ["https://docs.databricks.com/aws/en/machine-learning/"]
TXT_PATH = "/content/notes.txt"

# 3) Load documents
def load_docs():
    if SOURCE_TYPE == "pdf":
        loader = PyPDFLoader(PDF_PATH)
        return loader.load()
    elif SOURCE_TYPE == "url":
        loader = WebBaseLoader(URLS)
        return loader.load()
    elif SOURCE_TYPE == "txt":
        loader = TextLoader(TXT_PATH, encoding="utf-8")
        return loader.load()
    else:
        raise ValueError("Unsupported SOURCE_TYPE. Use 'pdf', 'url', or 'txt'.")

docs = load_docs()
print(f"✅ Loaded {len(docs)} document chunks (pre-split).")

# 4) Split into chunks
splitter = RecursiveCharacterTextSplitter(chunk_size=800, chunk_overlap=120)
splits = splitter.split_documents(docs)
print(f"✅ Split into {len(splits)} chunks.")

# 5) Create vector store with Chroma
emb = OpenAIEmbeddings(model="text-embedding-3-small")
SAVE_DIR = "/content/my_chroma_index_docs"

vs = Chroma.from_documents(
    documents=splits,
    embedding=emb,
    persist_directory=SAVE_DIR
)
vs.persist()

retriever = vs.as_retriever(search_kwargs={"k": 3})
print("✅ Chroma DB ready at:", SAVE_DIR)

# 6) Prompt with “use context only” + citations
prompt = ChatPromptTemplate.from_template(
    "You are a helpful assistant. Use ONLY the context to answer.\n\n"
    "Context:\n{context}\n\n"
    "Question: {question}\n\n"
    "If the answer is not in the context, say you don't know.\n"
    "At the end, list citations as [source#page_or_id]."
)

def format_docs(docs):
    lines = []
    for d in docs:
        src = d.metadata.get("source") or d.metadata.get("url") or "unknown"
        page = d.metadata.get("page", d.metadata.get("chunk_id", "na"))
        lines.append(f"[{src}#{page}] {d.page_content}")
    return "\n\n".join(lines)

# 7) LLM + chain
llm = ChatOpenAI(model="gpt-4o-mini", temperature=0.2)
rag_chain = (
    {"context": retriever | format_docs, "question": RunnablePassthrough()}
    | prompt
    | llm
    | StrOutputParser()
)

# 8) Q&A loop
print("\n💬 Ask about your document(s). Type 'exit' to quit.")
while True:
    q = input("Q: ").strip()
    if q.lower() in ["exit", "quit"]:
        print("👋 Goodbye!")
        break
    ans = rag_chain.invoke(q)
    print("A:", ans, "\n")


✅ Loaded 1 document chunks (pre-split).
✅ Split into 9 chunks.
✅ Chroma DB ready at: /content/my_chroma_index_docs

💬 Ask about your document(s). Type 'exit' to quit.
Q: what are foundation models
A: Foundation models commonly refer to large language models that have been trained over extensive datasets to be generally good at some tasks, such as chat, instruction following, and code generation. They serve as the base for layers of increasingly complex techniques in the development of production-quality GenAI applications. Foundation models generally fall under two categories: proprietary models (like GPT-3.5 and Gemini) and open source models (like Llama2-70B and DBRX) [source#4]. 

Q: what is databricks
A: Databricks is the data and AI company that provides a platform for unifying and democratizing data, analytics, and AI. It is relied upon by over 10,000 organizations worldwide, including major companies like Comcast, Condé Nast, and Grammarly, as well as over 50% of the Fortune 500

In [19]:
# RAG over PDF/URL/TXT using Pinecone DB (Cloud Hosted Managed Production Grade - Vector Database)

In [ ]:
# End of the Notebook

In [ ]:
# Build RAG Pipeline application via Gradio - UI.

In [ ]:
# Example - Gradio Implementation

In [ ]:
# ============================================================
# Gradio UI for a Tiny RAG over PDF / URL / TXT / Raw text
# Colab-ready
# ============================================================

!pip install -q gradio langchain-openai langchain-community langchain-text-splitters faiss-cpu pypdf bs4

import os, json, tempfile, textwrap
import gradio as gr

from google.colab import userdata
from langchain_openai import OpenAIEmbeddings, ChatOpenAI
from langchain_community.vectorstores import FAISS
from langchain_community.vectorstores.utils import DistanceStrategy
from langchain_text_splitters import RecursiveCharacterTextSplitter
from langchain_community.document_loaders import PyPDFLoader, TextLoader, WebBaseLoader
from langchain.schema import Document

# ────────────────────────────────────────────────────────────
# API key (expects you've set it with userdata.set(...) once)
# ────────────────────────────────────────────────────────────
os.environ["OPENAI_API_KEY"] = userdata.get("OPENAI_API_KEY")

# ────────────────────────────────────────────────────────────
# Helpers
# ────────────────────────────────────────────────────────────
def _format_docs(docs):
    """Format retrieved docs with simple citations."""
    lines = []
    for d in docs:
        src = d.metadata.get("source") or d.metadata.get("url") or "unknown"
        page = d.metadata.get("page", d.metadata.get("chunk_id", "na"))
        lines.append(f"[{src}#{page}] {d.page_content}")
    return "\n\n".join(lines)

def _citations(docs):
    """Return a compact citations string like [source#page], deduped."""
    tags = []
    seen = set()
    for d in docs:
        src = d.metadata.get("source") or d.metadata.get("url") or "unknown"
        page = d.metadata.get("page", d.metadata.get("chunk_id", "na"))
        tag = f"[{src}#{page}]"
        if tag not in seen:
            tags.append(tag)
            seen.add(tag)
    return " ".join(tags) if tags else "(no citations)"

# ────────────────────────────────────────────────────────────
# Build index: load → chunk → embed → FAISS (cosine)
# ────────────────────────────────────────────────────────────
def build_index(
    source_type,
    files,        # list of temp files from gradio for "Files"
    urls_text,    # str for "URL(s)" separated by newline/comma
    raw_text,     # str for "Raw Text"
    chunk_size,
    chunk_overlap,
    k,
    model_name,
    temperature
):
    try:
        # 1) Load docs
        docs = []
        if source_type == "Files (PDF/TXT)":
            if not files:
                return gr.update(value="Please upload at least one file."), None, None, None
            for f in files:
                # Decide by extension
                name = f.name if hasattr(f, "name") else str(f)
                if name.lower().endswith(".pdf"):
                    loader = PyPDFLoader(name)
                    docs.extend(loader.load())
                else:
                    loader = TextLoader(name, encoding="utf-8")
                    docs.extend(loader.load())

        elif source_type == "URL(s)":
            if not urls_text.strip():
                return gr.update(value="Please enter at least one URL."), None, None, None
            # split on newline or comma
            raw_urls = [u.strip() for u in urls_text.replace(",", "\n").split("\n") if u.strip()]
            loader = WebBaseLoader(raw_urls)
            docs = loader.load()

        elif source_type == "Raw Text":
            if not raw_text.strip():
                return gr.update(value="Please paste some text."), None, None, None
            docs = [Document(page_content=raw_text, metadata={"source": "raw.txt"})]

        else:
            return gr.update(value="Unsupported source type."), None, None, None

        if not docs:
            return gr.update(value="No text found after loading."), None, None, None

        # 2) Chunk
        splitter = RecursiveCharacterTextSplitter(
            chunk_size=int(chunk_size), chunk_overlap=int(chunk_overlap)
        )
        splits = splitter.split_documents(docs)

        # 3) Embeddings + FAISS (cosine)
        emb = OpenAIEmbeddings(model="text-embedding-3-small")
        vs = FAISS.from_documents(splits, embedding=emb, distance_strategy=DistanceStrategy.COSINE)
        retriever = vs.as_retriever(search_kwargs={"k": int(k)})

        # 4) LLM (GPT model)
        llm = ChatOpenAI(model=model_name, temperature=float(temperature))

        status = (
            f"✅ Index built.\n"
            f"- Chunks: {len(splits)}\n"
            f"- k: {k}\n"
            f"- LLM: {model_name} (temp={temperature})\n"
            f"- Similarity: COSINE (FAISS)"
        )
        return status, vs, retriever, llm

    except Exception as e:
        return f"❌ Error while building index: {e}", None, None, None

# ────────────────────────────────────────────────────────────
# Answer a question from the retriever context
# ────────────────────────────────────────────────────────────
def ask_question(chat_history, question, retriever, llm):
    if retriever is None or llm is None:
        chat_history = chat_history or []
        chat_history.append(("",
            "Please build the index first (choose a source and click 'Build Index')."
        ))
        return chat_history

    try:
        # Retrieve
        ctx_docs = retriever.get_relevant_documents(question)
        context = _format_docs(ctx_docs)

        # Prompt (simple & explicit grounding)
        messages = [
            {"role": "system",
             "content": "You are a helpful assistant. Use ONLY the provided context to answer. "
                        "If the answer is not present, say you don't know."},
            {"role": "user",
             "content": f"Context:\n{context}\n\nQuestion: {question}\nAnswer:"}
        ]
        resp = llm.invoke(messages).content

        # Append citations at the end
        cites = _citations(ctx_docs)
        final = f"{resp}\n\n**Citations:** {cites}"

        chat_history = chat_history or []
        chat_history.append((question, final))
        return chat_history

    except Exception as e:
        chat_history = chat_history or []
        chat_history.append((question, f"❌ Error: {e}"))
        return chat_history

# ────────────────────────────────────────────────────────────
# Gradio UI
# ────────────────────────────────────────────────────────────
with gr.Blocks(title="Tiny RAG UI (FAISS + OpenAI)") as app:
    gr.Markdown("## 🔎 Tiny RAG — Ask questions over your PDF/URL/TXT/Raw text")

    with gr.Accordion("1) Build Index", open=True):
        source_type = gr.Radio(
            ["Files (PDF/TXT)", "URL(s)", "Raw Text"],
            value="Files (PDF/TXT)",
            label="Source type"
        )
        files = gr.File(label="Upload PDF/TXT files", file_types=[".pdf", ".txt"], file_count="multiple", visible=True)
        urls_text = gr.Textbox(label="Enter URL(s) (comma- or newline‑separated)", visible=False, lines=3)
        raw_text = gr.Textbox(label="Paste Raw Text", visible=False, lines=6, placeholder="Paste your text here...")

        with gr.Row():
            chunk_size = gr.Number(label="Chunk size", value=800)
            chunk_overlap = gr.Number(label="Chunk overlap", value=120)
            k = gr.Slider(1, 10, value=3, step=1, label="Top‑k retrieved")

        with gr.Row():
            model_name = gr.Textbox(label="GPT model", value="gpt-4o-mini")
            temperature = gr.Slider(0.0, 1.0, value=0.2, step=0.05, label="Temperature")

        build_btn = gr.Button("🔨 Build Index")
        status = gr.Markdown("Status will appear here.")

    with gr.Accordion("2) Ask Questions", open=True):
        chat = gr.Chatbot(height=320, label="Chat")
        question = gr.Textbox(label="Your question")
        ask_btn = gr.Button("Ask")

    # App state
    vs_state = gr.State(None)
    retriever_state = gr.State(None)
    llm_state = gr.State(None)

    # Source type toggling
    def toggle_inputs(src):
        return (
            gr.update(visible=(src == "Files (PDF/TXT)")),
            gr.update(visible=(src == "URL(s)")),
            gr.update(visible=(src == "Raw Text")),
        )

    source_type.change(
        toggle_inputs, inputs=[source_type], outputs=[files, urls_text, raw_text]
    )

    # Build index
    build_btn.click(
        build_index,
        inputs=[source_type, files, urls_text, raw_text, chunk_size, chunk_overlap, k, model_name, temperature],
        outputs=[status, vs_state, retriever_state, llm_state]
    )

    # Ask
    ask_btn.click(
        ask_question,
        inputs=[chat, question, retriever_state, llm_state],
        outputs=[chat]
    )

# Launch (share=True gives a public URL from Colab)
app.launch(share=True)


   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.5/2.5 MB 27.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 31.3/31.3 MB 33.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 313.2/313.2 kB 18.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 45.2/45.2 kB 2.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 50.9/50.9 kB 2.9 MB/s eta 0:00:00


/tmp/ipython-input-2-3473171679.py:192: UserWarning: You have not specified a value for the `type` parameter. Defaulting to the 'tuples' format for chatbot messages, but this is deprecated and will be removed in a future version of Gradio. Please set type='messages' instead, which uses openai-style dictionaries with 'role' and 'content' keys.
  chat = gr.Chatbot(height=320, label="Chat")


Colab notebook detected. To show errors in colab notebook, set debug=True in launch()
* Running on public URL: https://a6c40ec6302667a6da.gradio.live

This share link expires in 1 week. For free permanent hosting and GPU upgrades, run `gradio deploy` from the terminal in the working directory to deploy to Hugging Face Spaces (https://huggingface.co/spaces)


In [ ]:
# End of the notebook